In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('sentiment_scored_headlines_FinBERT.csv')

In [4]:
# Create weight mapping
df['weight'] = np.where(df['sentiment_label'] == 'Positive', 1,
               np.where(df['sentiment_label'] == 'Negative', -1, 0))

In [12]:
# Calculate weighted scores
df['weighted_score'] = df['sentiment_score'] * df['weight']
df['abs_weight'] = abs(df['weight'])

In [29]:
# Group by stock and date. The sort=False is to ensure that groupby keeps the rows in the same order they were in.
grouped = df.groupby(['stock', 'date', 'open', 'close'], sort=False).agg(
    total_weighted=('weighted_score', 'sum'),
    total_abs_weight=('abs_weight', 'sum')
).reset_index()

In [30]:
# Calculate final sentiment score with error handling
grouped['sentiment_score'] = np.where(
    grouped['total_abs_weight'] > 0,
    grouped['total_weighted'] / grouped['total_abs_weight'],
    0  # Handle neutral-only days
)

In [32]:
# Create final dataframe and save
final_df = grouped[['stock', 'date', 'open', 'close', 'sentiment_score']]
final_df.to_csv('average_sentiment_scores.csv', index=False)

In [33]:
print(final_df.head(10))

  stock        date     open    close  sentiment_score
0  aapl  2025-03-03  241.790  238.030         0.000002
1  aapl  2025-02-28  236.950  241.840        -0.196184
2  aapl  2025-02-27  239.410  237.300        -0.990303
3  aapl  2025-02-26  244.330  240.360         0.926968
4  aapl  2025-02-25  248.000  247.040         0.999885
5  aapl  2025-02-24  244.925  247.172        -0.347224
6  aapl  2025-02-23  248.000  247.040         0.952590
7  aapl  2025-02-21  245.950  245.550         0.000000
8  aapl  2025-02-20  244.940  245.830        -0.922531
9  aapl  2025-02-19  244.660  244.870         0.761562
